## Caption classification ##

This model can be used to train and classify on any differentiation texts. 

In [4]:
import helper_2
import glob

In [5]:
files = glob.glob('/home/ubuntu/Thesis/Consolidated Data/*.txt')
sentences = []

for file in files:
    with open(file) as texts:
        content = texts.readlines()
        for text in content: 
            sentences.append(text)

In [6]:
texts = ''.join(sentences)
print(*sentences[:1],sep='\n')
print(type(texts))
print(len(texts))
print(texts[:15])

That's how we roll.

<class 'str'>
9230544
That's how we r


In [7]:
import pandas as pd

In [8]:
pandas = pd.DataFrame({'sentences':sentences})

In [9]:
pandas.head()

,sentences
0,That's how we roll.\n
1,It’s true! He calls me His co-pilot.\n
2,I got it at a garage sale.\n
3,"You wanted a bigger halo, yes it is razor shar..."
4,You had me at Halo.\n


In [10]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def load_clean_descriptions(filename):
    doc = load_doc(filename)
    descriptions = []
    for line in doc.split():
#         tokens = line.split()
        image_desc = line
        descriptions.append(image_desc)
    return descriptions

In [11]:
train_descriptions = load_clean_descriptions('/home/ubuntu/Thesis/8k_descriptions.txt')

In [12]:
check_out = pd.read_table('/home/ubuntu/Thesis/Flickr8k.token.txt', sep='\t', names=('A', 'B'))

In [13]:
check_out.head()

,A,B
0,1000268201_693b08cb0e.jpg#0,A child in a pink dress is climbing up a set o...
1,1000268201_693b08cb0e.jpg#1,A girl going into a wooden building .
2,1000268201_693b08cb0e.jpg#2,A little girl climbing into a wooden playhouse .
3,1000268201_693b08cb0e.jpg#3,A little girl climbing the stairs to her playh...
4,1000268201_693b08cb0e.jpg#4,A little girl in a pink dress going into a woo...


In [14]:
pandas_2 = check_out.drop(columns='A')

In [15]:
pandas_2.head()

,B
0,A child in a pink dress is climbing up a set o...
1,A girl going into a wooden building .
2,A little girl climbing into a wooden playhouse .
3,A little girl climbing the stairs to her playh...
4,A little girl in a pink dress going into a woo...


In [19]:
pandas_2['value'] = 0

In [20]:
pandas['value'] = 1

In [21]:
len(pandas), len(pandas_2)

(164995, 40460)

In [22]:
pandas.head()

,sentences,value
0,That's how we roll.\n,1
1,It’s true! He calls me His co-pilot.\n,1
2,I got it at a garage sale.\n,1
3,"You wanted a bigger halo, yes it is razor shar...",1
4,You had me at Halo.\n,1


In [23]:
pandas_2.head()

,B,value
0,A child in a pink dress is climbing up a set o...,0
1,A girl going into a wooden building .,0
2,A little girl climbing into a wooden playhouse .,0
3,A little girl climbing the stairs to her playh...,0
4,A little girl in a pink dress going into a woo...,0


In [24]:
pandas_2 = pandas_2.rename(index=str, columns={"B" : "sentences"})

In [25]:
pandas_entire = pd.concat([pandas[:40460], pandas_2])

In [26]:
pandas_entire.head()

,sentences,value
0,That's how we roll.\n,1
1,It’s true! He calls me His co-pilot.\n,1
2,I got it at a garage sale.\n,1
3,"You wanted a bigger halo, yes it is razor shar...",1
4,You had me at Halo.\n,1


In [27]:
len(pandas_entire)

80920

In [28]:
import string
import re
re_tok = re.compile('([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
def tokenize(s): return s.split()

In [29]:
def clean_descriptions(descriptions):
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    for _, desc_list in descriptions.items():
        for i in range(len(desc_list)):
            desc = desc_list[i]
            desc = desc.split()
            desc = [word.lower() for word in desc]
            desc = [re_punc.sub('', w) for w in desc]
            desc = [word for word in desc if len(word)>1]
            desc = [word for word in desc if word.isalpha()]
            desc_list[i] =  ' '.join(desc)

In [30]:
pandas_entire['sentences'] = pandas_entire['sentences'].str.replace('\W+', ' ')

In [31]:
pandas_entire.head()

,sentences,value
0,That s how we roll,1
1,It s true He calls me His co pilot,1
2,I got it at a garage sale,1
3,You wanted a bigger halo yes it is razor sharp,1
4,You had me at Halo,1


# Word2vec train

In [32]:
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [33]:
from nltk.tokenize.toktok import ToktokTokenizer
toktok = ToktokTokenizer()

In [34]:
tokenized_sentences = [toktok.tokenize(sent) for sent in pandas_entire['sentences']]

In [35]:
# tokenized_sentences[:10]

In [36]:
(len(pandas_entire['sentences']))

80920

In [37]:
word2vec_model = gensim.models.Word2Vec(tokenized_sentences, min_count=1, size=300, workers=4)


2018-04-02 19:35:31,374 : INFO : collecting all words and their counts
2018-04-02 19:35:31,375 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-04-02 19:35:31,403 : INFO : PROGRESS: at sentence #10000, processed 97471 words, keeping 9349 word types
2018-04-02 19:35:31,435 : INFO : PROGRESS: at sentence #20000, processed 202854 words, keeping 15345 word types
2018-04-02 19:35:31,469 : INFO : PROGRESS: at sentence #30000, processed 313253 words, keeping 19478 word types
2018-04-02 19:35:31,503 : INFO : PROGRESS: at sentence #40000, processed 422801 words, keeping 22930 word types
2018-04-02 19:35:31,533 : INFO : PROGRESS: at sentence #50000, processed 531751 words, keeping 24923 word types
2018-04-02 19:35:31,563 : INFO : PROGRESS: at sentence #60000, processed 639798 words, keeping 25946 word types
2018-04-02 19:35:31,593 : INFO : PROGRESS: at sentence #70000, processed 746558 words, keeping 26867 word types
2018-04-02 19:35:31,623 : INFO : PROGRESS: at se

In [40]:
# word2vec_model = gensim.models.Word2Vec(min_count=1, size=500, workers=4, iter = 50)

In [94]:
# word2vec_model.wv.save_word2vec_format('/home/ubuntu/Thesis/new_classification')

2018-03-23 02:50:25,421 : INFO : storing 27603x300 projection weights into /home/ubuntu/Thesis/new_classification


In [41]:
# word2vec_model.wv.load_word2vec_format('/home/ubuntu/Thesis/classification/')

In [42]:
import keras

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

tokenizer_keras = Tokenizer()

tokenizer_keras.fit_on_texts(pandas_entire['sentences'])

from keras.preprocessing.sequence import pad_sequences

sequences_keras = tokenizer_keras.texts_to_sequences(pandas_entire['sentences'])

padded_seq_keras = pad_sequences(sequences_keras)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [43]:
print('padded_seq_shape', padded_seq_keras.shape)

padded_seq_shape (80920, 51)


In [1]:
# word2vec_model.wv['listen']

In [45]:
import sklearn

from sklearn.model_selection import KFold

x = padded_seq_keras
y = pandas_entire['value']

In [46]:
print(y.shape, x.shape)

(80920,) (80920, 51)


In [47]:


word_index = tokenizer_keras.word_index


vocab_size = len(word_index)
print(vocab_size)

embedding_size = 300

22796


In [48]:
import numpy as np
embedding_matrix = np.zeros((vocab_size+1, embedding_size))

In [49]:
for word,i in word_index.items():
    if word in word2vec_model:
        embedding_vector = word2vec_model[word]
    else:
        embedding_vector = None
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


# Below is the CNN model to classify texts. Its bidirectional model. Super cool results.

In [50]:
import keras.backend as K
from keras.models import *
from keras.layers import Input, merge, Conv2D, MaxPooling1D, MaxPooling2D, UpSampling1D, Dropout, Cropping2D, Conv1D, GlobalMaxPooling1D
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.layers import Embedding, Dense, Flatten
from keras.layers import Merge
from keras.layers import LSTM, Bidirectional, Conv1D, concatenate, Permute, Dropout



In [52]:
input_length_variable = padded_seq_keras.shape[1]
seed = 0

import pdb

convs = []
filter_sizes = [2,3]


def create_cnn(include_top = True, weights= None):
    
    sequence_input = Input(shape=(input_length_variable, ), dtype='int32', name='input')
    embedding_layer = Embedding(vocab_size + 1, embedding_size, weights= [embedding_matrix],
                                input_length= input_length_variable,
                                name = 'embedding', trainable=True)(sequence_input)
    pdb.set_trace()
    print(embedding_layer.shape)
    print(sequence_input.shape)
    
    
    for fsz in filter_sizes:
        l_conv = Conv1D(nb_filter=2,filter_length=fsz,activation='relu')(embedding_layer)
        lstm = Bidirectional(LSTM(1, return_sequences = True))(l_conv)
        l_pool = GlobalMaxPooling1D()(lstm)
#         lstm = (LSTM(256))(l_pool)
        convs.append(l_pool)
        
#     nn = Concatenate(name='features')(convs)
    l_merge = Merge(mode='concat', concat_axis=1)(convs)
#     l_cov1= Conv1D(128, 5, activation='relu')(l_merge)
#     l_pool1 = MaxPooling1D(5)(l_cov1)
#     l_cov2 = Conv1D(256, 5, activation='relu')(l_pool1)
#     l_pool2 = MaxPooling1D(5)(l_cov2)
#     l_flat = Flatten()(l_pool2)
#     l_dense = Dense(128, activation='relu')(l_flat)
    
    if include_top:
        nn = Dropout(0.5, seed=seed, name='dropout')(l_merge)
        weights_np = Dense(nn)
        weights_np = weights_np.get_weights()
        predictions = Dense(1, init='normal', name='output')(nn)
        model = Model(sequence_input, predictions)
    else:
        model = Model(sequence_input, nn)
        
    return model

In [53]:
keras_model = create_cnn()

> <ipython-input-52-e6cc26e33dca>(17)create_cnn()
-> print(embedding_layer.shape)
(Pdb) n
(?, 51, 300)
> <ipython-input-52-e6cc26e33dca>(18)create_cnn()
-> print(sequence_input.shape)
(Pdb) c
(?, 51)


/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=2, kernel_size=2)`
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=2, kernel_size=3)`
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, name="output", kernel_initializer="normal")`


In [139]:
keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 51)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 51, 300)      6839100     input[0][0]                      
__________________________________________________________________________________________________
conv1d_20 (Conv1D)              (None, 50, 2)        1202        embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_21 (Conv1D)              (None, 49, 2)        1802        embedding[0][0]                  
__________________________________________________________________________________________________
bidirectio

In [140]:
from sklearn.model_selection import KFold
n_splits = 3

kf = KFold(n_splits=n_splits)
kf.get_n_splits(3)

3

In [141]:
def rmsle(y_true, y_pred):
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)

### 99.5% accuracy

In [142]:
# train cnn
keras_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'acc'])

batch_size = 128
epochs = 5
for train_index, val_index in kf.split(x):
    x_train, x_val = x[train_index], x[val_index]
    y_train, y_val = y[train_index], y[val_index]
    print(x_train.shape, y_train.shape)
    keras_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=batch_size)

(53946, 51) (53946,)
Train on 53946 samples, validate on 26974 samples
Epoch 1/5
53946/53946 [==============================] - 207s 4ms/step - loss: 0.1161 - mean_absolute_error: 0.2177 - acc: 0.8529 - val_loss: 0.1349 - val_mean_absolute_error: 0.3347 - val_acc: 0.8840
Epoch 2/5
53946/53946 [==============================] - 206s 4ms/step - loss: 0.0546 - mean_absolute_error: 0.1576 - acc: 0.9331 - val_loss: 0.0807 - val_mean_absolute_error: 0.2424 - val_acc: 0.9209
Epoch 3/5
53946/53946 [==============================] - 206s 4ms/step - loss: 0.0483 - mean_absolute_error: 0.1492 - acc: 0.9446 - val_loss: 0.0768 - val_mean_absolute_error: 0.2394 - val_acc: 0.9299
Epoch 4/5
53946/53946 [==============================] - 202s 4ms/step - loss: 0.0455 - mean_absolute_error: 0.1453 - acc: 0.9479 - val_loss: 0.0756 - val_mean_absolute_error: 0.2383 - val_acc: 0.9356
Epoch 5/5
53946/53946 [==============================] - 195s 4ms/step - loss: 0.0457 - mean_absolute_error: 0.1458 - acc: 0.

In [143]:
# model_1 = keras_model.save_weights('/home/ubuntu/Thesis/new_classification.h5')
# model_1_1 = keras_model.save('/home/ubuntu/Thesis/new_classification_2.h5')

# Train model here

In [ ]:
# # train cnn
# def mean_pred(y_true, y_pred):
#     return K.mean(y_pred)

# keras_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', mean_pred])

# batch_size = 128
# epochs = 1
# for train_index, val_index in kf.split(x):
#     x_train, x_val = x[train_index], x[val_index]
#     y_train, y_val = y[train_index], y[val_index]
#     print(x_train.shape, y_train.shape)
#     keras_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=epochs, batch_size=batch_size)

In [ ]:
# model_2 = keras_model.save_weights('/home/ubuntu/Thesis/new_classification_second.h5')
# model_2_1 = keras_model.save('/home/ubuntu/Thesis/new_classification_second_2.h5')

In [312]:
check = ['so you’d like it who’s bigger ideas']

In [313]:
from keras.preprocessing.text import Tokenizer
text = pandas_entire['sentences'].astype(str).tolist()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

In [314]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

In [315]:
sequences_check = tokenizer.texts_to_sequences(check)

In [316]:
x_test = pad_sequences(sequences_check, maxlen=51)

# Load model here 

In [317]:
# model_one = load_model('/home/ubuntu/Thesis/new_classification_2.h5')

In [318]:
y_test = model_one.predict(x_test)

In [319]:
print(y_test)

[[0.92933923]]
